<a href="https://colab.research.google.com/github/Raju513-dot/Fmml_module/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Yes, averaging the validation accuracy across multiple splits, typically in the context of cross-validation, generally gives more consistent and reliable results. Here's why:

Reduces Variance: Validation performance on a single split can be highly variable due to the randomness in the selection of training and validation sets. By averaging across multiple splits, you reduce the impact of outliers or specific characteristics of any single split.

Better Generalization: Different splits expose the model to a more diverse range of training and validation examples. As a result, the average accuracy reflects the model's ability to generalize across various subsets of the data.

Minimizes Overfitting: Averaging the accuracy across multiple splits ensures that the performance isn't biased by the peculiarities of a specific dataset split, which could lead to overfitting to that particular split.

Stability in Evaluation: With multiple splits, you can get a more stable and robust estimate of the model’s performance, especially when the dataset is small or imbalanced.

In short, averaging validation accuracy across splits in techniques like k-fold cross-validation is a standard approach to ensure a more accurate and consistent evaluation of model performance.
2) Yes, averaging validation accuracy across multiple splits, such as in k-fold cross-validation, typically provides a more accurate estimate of test accuracy. Here's why:1. Reduces Bias:
When you use a single train-validation split, the model's performance might be biased based on that particular split. If the split is not representative of the overall data distribution, it can lead to an inaccurate estimate of how well the model will perform on unseen data. Multiple splits help mitigate this bias, providing a more reliable estimate of test accuracy.
2. More Representative of Real-World Performance:
By training and validating on different portions of the dataset across multiple splits, the model is exposed to different data points in each split. This simulates multiple scenarios, allowing the model’s performance to be assessed across a broader range of conditions, which better approximates real-world performance on unseen test data.
3. Decreases Variability:
A single validation split can give a performance estimate with high variability due to random factors in how the split was made. Averaging results across multiple splits smooths out this variability and gives a more stable and robust estimate of test accuracy.
4. Better Use of Data:
In cross-validation, the model is trained and validated on different portions of the dataset multiple times, meaning all data points are used for both training and validation across different iterations. This makes better use of the available data, especially if the dataset is small, leading to a more accurate estimate of test accuracy.
5. Less Overfitting to Validation Set:
When you average validation performance across multiple splits, the model isn't tuned to a single validation set. This reduces the risk of overfitting to the validation set and provides a better reflection of how the model will perform on unseen test data.
Caveat:
While cross-validation gives a more reliable estimate of test accuracy, the performance on the actual test set may still vary, especially if the test set comes from a slightly different distribution than the training/validation set. However, cross-validation significantly improves the reliability of performance estimates compared to a single validation split.
3) ChatGPT Auto
You said:
Does averaging the validation accuracy across multiple splits give more consistent results?
ChatGPT said:
ChatGPT
Yes, averaging the validation accuracy across multiple splits, typically in the context of cross-validation, generally gives more consistent and reliable results. Here's why:
Reduces Variance: Validation performance on a single split can be highly variable due to the randomness in the selection of training and validation sets. By averaging across multiple splits, you reduce the impact of outliers or specific characteristics of any single split.
Better Generalization: Different splits expose the model to a more diverse range of training and validation examples. As a result, the average accuracy reflects the model's ability to generalize across various subsets of the data.
Minimizes Overfitting: Averaging the accuracy across multiple splits ensures that the performance isn't biased by the peculiarities of a specific dataset split, which could lead to overfitting to that particular split.
Stability in Evaluation: With multiple splits, you can get a more stable and robust estimate of the model’s performance, especially when the dataset is small or imbalanced.
In short, averaging validation accuracy across splits in techniques like k-fold cross-validation is a standard approach to ensure a more accurate and consistent evaluation of model performance.
You said:
Does it give more accurate estimate of test accuracy?
ChatGPT said:
ChatGPT
Yes, averaging validation accuracy across multiple splits, such as in k-fold cross-validation, typically provides a more accurate estimate of test accuracy. Here's why:
1. Reduces Bias:
When you use a single train-validation split, the model's performance might be biased based on that particular split. If the split is not representative of the overall data distribution, it can lead to an inaccurate estimate of how well the model will perform on unseen data. Multiple splits help mitigate this bias, providing a more reliable estimate of test accuracy.
2. More Representative of Real-World Performance:
By training and validating on different portions of the dataset across multiple splits, the model is exposed to different data points in each split. This simulates multiple scenarios, allowing the model’s performance to be assessed across a broader range of conditions, which better approximates real-world performance on unseen test data.
3. Decreases Variability:
A single validation split can give a performance estimate with high variability due to random factors in how the split was made. Averaging results across multiple splits smooths out this variability and gives a more stable and robust estimate of test accuracy.
4. Better Use of Data:
In cross-validation, the model is trained and validated on different portions of the dataset multiple times, meaning all data points are used for both training and validation across different iterations. This makes better use of the available data, especially if the dataset is small, leading to a more accurate estimate of test accuracy.
5. Less Overfitting to Validation Set:
When you average validation performance across multiple splits, the model isn't tuned to a single validation set. This reduces the risk of overfitting to the validation set and provides a better reflection of how the model will perform on unseen test data.
Caveat:
While cross-validation gives a more reliable estimate of test accuracy, the performance on the actual test set may still vary, especially if the test set comes from a slightly different distribution than the training/validation set. However, cross-validation significantly improves the reliability of performance estimates compared to a single validation split.
You said:
What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
ChatGPT said:
ChatGPT
Yes, increasing the number of iterations in cross-validation or similar techniques can lead to a better and more reliable estimate of test accuracy, but there are diminishing returns. Here's how the number of iterations (or folds in k-fold cross-validation) affects the estimate:
1. More Iterations Reduce Variance:Higher number of iterations (i.e., more folds in k-fold cross-validation) generally reduces the variance in your performance estimate. Each iteration uses a different subset of the data for training and validation, so averaging results over more iterations captures more information about how the model performs across different data subsets.For instance, if you use 10-fold cross-validation compared to 5-fold, you are training and validating on 10 different splits rather than 5, which gives a more stable estimate.2. Better Data Utilization:With more iterations, you make better use of the data. In leave-one-out cross-validation (LOO-CV), for example, each iteration trains the model on all the data except for one sample, leading to the maximal use of the dataset for training. This often results in a more accurate estimate because each data point gets a chance to be in the validation set.However, the computational cost of extreme cases like LOO-CV can be high.3. Diminishing Returns:More iterations do improve the estimate, but after a certain point, the improvements are marginal. For example, moving from 5-fold to 10-fold cross-validation provides a noticeable improvement in stability and accuracy of the estimate, but moving from 10-fold to 20-fold may only offer slight gains in precision.This is because you are already averaging over a good portion of the data, and additional folds are unlikely to provide new information unless the dataset is highly variable or small.4. Impact of Data Size:If the dataset is small, using more iterations (e.g., 10-fold or LOO-CV) can significantly improve the reliability of the estimate because the model will be validated on more data points.For large datasets, using more iterations provides less additional benefit because the model's performance will already be consistent across large data subsets.5. Computational Trade-offs:More iterations require more computation. While increasing iterations gives a more robust estimate, it also increases computational cost. For very large datasets or models with high training time, there’s a practical limit to how many iterations you can run before the costs outweigh the benefits.A good balance (e.g., 5-fold or 10-fold cross-validation) is often chosen to get a reliable estimate without excessive computation.Summary:Increasing the number of iterations typically results in a better, more stable estimate of test accuracy by reducing variance and making better use of the data.However, the gains from additional iterations diminish after a certain point, and you need to balance the improvement in accuracy against the computational cost.
4) Increasing the number of iterations in cross-validation can help mitigate the issues that arise from having a very small training or validation dataset, but it does not fully solve them. Here’s an analysis of how higher iterations affect small datasets and the inherent challenges:
1. More Iterations (Higher k in k-Fold CV) Helps Use Data More Efficiently:When you have a very small dataset, increasing the number of iterations (e.g., using 10-fold cross-validation instead of 5-fold) allows the model to be trained and validated on more combinations of the data.This ensures that the model gets trained on nearly the entire dataset at least once, which can reduce the variance of the estimate and give a better approximation of test accuracy.In extreme cases like leave-one-out cross-validation (LOO-CV), each data point is used for validation once, and the model is trained on the rest, maximizing data utilization.2. Benefits of Increasing Iterations:More Training Data: As the number of iterations increases, the model trains on a larger portion of the data in each split. For example, in 5-fold CV, the model trains on 80% of the data, but in 10-fold CV, it trains on 90%. In LOO-CV, the model trains on all data except one point.Less Overfitting to the Validation Set: More iterations reduce the risk that the model is overfitting to a small validation set, which can be a risk with a very small dataset.Stability in Performance Estimation: With more iterations, especially in small datasets, the performance estimate becomes more robust because every data point is used for both training and validation, reducing the likelihood of a split that is particularly unrepresentative.3. Challenges with Very Small Datasets:Limited Generalization: Despite increasing the number of iterations, the fundamental problem with very small datasets is that the model has a limited amount of data to learn from. Even though cross-validation helps maximize data usage, it doesn’t increase the actual size of the training data.If the dataset is too small, the model might overfit to the small training sets in each fold, meaning the performance in cross-validation might still be overly optimistic and may not generalize well to unseen data.High Variance Models: Small datasets often lead to models with high variance because there aren’t enough examples for the model to learn robust patterns. Even with more iterations, if the dataset is too small, the model may still struggle to generalize well.4. Diminishing Returns:As mentioned earlier, while increasing the number of iterations helps, the gains start to diminish after a certain point. For example, moving from 10-fold to LOO-CV might slightly improve the performance estimate, but the computational cost increases significantly, while the benefit may not justify it.With a very small dataset, increasing iterations may give more stable estimates of accuracy, but the fundamental problem of insufficient data to learn meaningful patterns remains.5. Alternatives for Small Datasets:Data Augmentation: In cases like image or text data, you can use data augmentation techniques to artificially increase the size of your training dataset, giving the model more varied data to learn from.Transfer Learning: Pretrained models (especially in domains like computer vision or NLP) can be fine-tuned on small datasets, leveraging large-scale learning from other tasks.Regularization: To prevent overfitting on small datasets, use regularization techniques like dropout, weight decay, or early stopping.Synthetic Data: In some cases, generating synthetic data or using techniques like SMOTE (Synthetic Minority Over-sampling Technique) can help when you have a small or imbalanced dataset.Conclusion:
Increasing the number of iterations (e.g., using k-fold cross-validation with a higher k or LOO-CV) helps make better use of a small dataset and gives a more reliable estimate of test accuracy. However, it does not overcome the fundamental limitations of having a very small training or validation dataset. To effectively deal with very small datasets, it’s often necessary to supplement cross-validation with techniques like data augmentation, transfer learning, or regularization to improve the model’s ability to generalize.